In [2]:
import cv2
from ultralytics import YOLO

# Charger le modèle YOLOv8 (remplacez par votre modèle personnalisé si nécessaire)
model = YOLO('yolov8l.pt')  # Remplacez par votre modèle personnalisé si nécessaire

# Charger la vidéo
video_path = 'penalty1.mp4'
cap = cv2.VideoCapture(video_path)

# Vérifiez si la vidéo est ouverte correctement
if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

# Obtenir les dimensions de la vidéo pour l'enregistrement
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

# Créer un fichier vidéo de sortie pour enregistrer les résultats annotés
out = cv2.VideoWriter('output_video_test.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Paramètre de confiance (threshold) pour les détections
confidence_threshold = 0.3  # Ajustez selon vos besoins

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Faire une prédiction sur le frame avec le seuil de confiance ajusté
    results = model(frame, conf=confidence_threshold)
    print(results)

    # Annoter l'image avec les résultats
    annotated_frame = results[0].plot()

    # Afficher la frame annotée
    cv2.imshow('Rugby Ball Detection', annotated_frame)

    # Enregistrer la frame annotée dans le fichier vidéo
    out.write(annotated_frame)

    # Pause de 25ms entre chaque frame (40 pour 25fps, 25 pour 40fps)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

100%|██████████| 83.7M/83.7M [00:23<00:00, 3.72MB/s]


Dimensions de la vidéo : 1920x1080, FPS : 30

0: 384x640 8 persons, 358.1ms
Speed: 2.1ms preprocess, 358.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banan

In [8]:
import cv2
import numpy as np

# Charger la vidéo
video_path = 'penalty5.mp4'
cap = cv2.VideoCapture(video_path)

# Vérifiez si la vidéo est ouverte correctement
if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

# Obtenir les dimensions de la vidéo pour l'enregistrement
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

# Créer un fichier vidéo de sortie pour enregistrer les résultats annotés
output_path = 'output_with_annotations.mp4'  # Changer l'extension pour .avi
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Utiliser XVID comme codec
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
prev_frame = None

while True:
    # Lire la frame actuelle
    ret, frame = cap.read()

    # Si la vidéo est terminée, quitter
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir en niveaux de gris pour simplifier la détection de variations
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        # Calculer la différence absolue entre la frame actuelle et la précédente
        diff = cv2.absdiff(prev_frame, gray_frame)

        # Appliquer un seuil pour créer une image binaire (pour détecter des changements significatifs)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Trouver les contours des zones où il y a des différences
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Dessiner les contours sur la frame originale (en couleur)
        annotated_frame = frame.copy()
        cv2.drawContours(annotated_frame, contours, -1, (0, 255, 0), 2)

        # Si une variation importante est détectée, ajouter une annotation texte
        if np.sum(diff) > 5000:  # seuil arbitraire
            cv2.putText(annotated_frame, f"Variation detected at frame {frame_count}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Enregistrer la frame annotée dans le fichier vidéo
        out.write(annotated_frame)

    # Mettre à jour la frame précédente
    prev_frame = gray_frame
    frame_count += 1

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processed and saved successfully.")


Dimensions de la vidéo : 1920x1080, FPS : 30
Fin de la vidéo ou erreur de lecture.
Video processed and saved successfully.


In [6]:

from roboflow import Roboflow

# Initialiser Roboflow avec l'API key
rf = Roboflow(api_key="G4CrZeGMRtBn93ZkKN1t")

# Accéder au projet et au modèle
project = rf.workspace().project("rugby-vision")
model = project.version("3").model

# Faire des prédictions sur la vidéo
job_id, signed_url, expire_time = model.predict_video(
    "test1.mp4",  # Remplacez par le nom de votre vidéo
    fps=5,        # Cadence en images par seconde
    prediction_type="batch-video",
)

# Poller jusqu'à ce que les résultats de la vidéo soient disponibles
results = model.poll_until_video_results(job_id)

# Afficher les résultats
print(results)


loading Roboflow workspace...
loading Roboflow project...
Checking for video inference results for job 2e6e6445-697f-4146-b81c-4357b74ef694 every 60s
(0s): Checking for inference results
{'frame_offset': [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306, 312, 318, 324, 330, 336, 342, 348, 354, 360, 366, 372, 378, 384, 390, 396, 402, 408, 414, 420, 426, 432, 438, 444, 450, 456, 462, 468, 474, 480, 486, 492, 498, 504, 510], 'time_offset': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0, 7.2, 7.4, 7.6, 7.8, 8.0, 8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.6, 12.8, 13.0, 13.2, 13.4, 13.6, 13.

In [7]:
# Initialiser Roboflow avec l'API key
rf = Roboflow(api_key="G4CrZeGMRtBn93ZkKN1t")

# Accéder au projet et au modèle
project = rf.workspace().project("rugby-vision")
model = project.version("3").model

# Faire des prédictions sur la vidéo
job_id, signed_url, expire_time = model.predict_video(
    "test1.mp4",  # Remplacez par le nom de votre vidéo
    fps=5,        # Cadence en images par seconde
    prediction_type="batch-video",
)

# Poller jusqu'à ce que les résultats de la vidéo soient disponibles
results = model.poll_until_video_results(job_id)

# Afficher les résultats
print(results)

# Si les résultats contiennent une URL de la vidéo annotée, vous pouvez la télécharger
if "output" in results and "url" in results["output"]:
    annotated_video_url = results["output"]["url"]
    print("Video annotated available at:", annotated_video_url)

    # Optionnel : pour télécharger la vidéo annotée
    import requests
    response = requests.get(annotated_video_url)

    with open("annotated_video.mp4", "wb") as f:
        f.write(response.content)
    print("Annotated video downloaded as 'annotated_video.mp4'")
else:
    print("No annotated video found.")


loading Roboflow workspace...
loading Roboflow project...
Checking for video inference results for job 39a77bfa-4220-4c5e-b632-49c2462b31d6 every 60s
(0s): Checking for inference results
{'frame_offset': [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306, 312, 318, 324, 330, 336, 342, 348, 354, 360, 366, 372, 378, 384, 390, 396, 402, 408, 414, 420, 426, 432, 438, 444, 450, 456, 462, 468, 474, 480, 486, 492, 498, 504, 510], 'time_offset': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0, 7.2, 7.4, 7.6, 7.8, 8.0, 8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.6, 12.8, 13.0, 13.2, 13.4, 13.6, 13.

In [1]:
# Installez les dépendances nécessaires si vous ne les avez pas encore
!pip install torch torchvision
!pip install -U roboflow

from roboflow import Roboflow

# Charger votre dataset
rf = Roboflow(api_key="G4CrZeGMRtBn93ZkKN1t")
project = rf.workspace().project("rugby-vision")
dataset = project.version(3).download("yolov5")  # ou le format que vous souhaitez

# Vérifiez que les données sont bien téléchargées
print(dataset)

# Clonez le dépôt YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt




loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to rugby-vision-3 in yolov5pytorch:: 100%|██████████| 290/290 [00:00<00:00, 819.07it/s]

c:\Users\marti\OneDrive - Fondation EPF\Bureau\5ème_année\Projet 5A\P5A_Rugby_Vision\yolov5


Cloning into 'yolov5'...
This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.


In [6]:
!python train.py --img 640 --batch 16 --epochs 2 --data "rugby-vision-3/data.yaml" --weights 'yolov5s.pt'


train: weights='yolov5s.pt', cfg=, data=rugby-vision-3/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=2, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
unknown option: -
usage: git [-v | --version] [-h | --help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]


In [1]:
from ultralytics import YOLO


model = YOLO('data.yaml')
model.train(data='data.yaml', epochs=2, imgsz=640, batch=16, device='cpu')

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


KeyError: 'backbone'